# Dealing with camera geometries in GammaLearn and for LST-1

The camera geometry is naturally different in MC and real data.

While the real data geometry is now dealt with in `ctapipe_io_lst` during data reading, differences remain that lead to wrong reconstruction.

Here is a summary done for a bugfix in June  2023.

In [ ]:
import ctapipe

print(f"ctapipe version {ctapipe.__version__}")

In [ ]:
import numpy as np
from ctapipe.instrument import CameraGeometry, SubarrayDescription
from ctapipe.visualization import CameraDisplay

In [ ]:
# dl1_mc_path = "/Users/thomasvuillaume/Work/CTA/Data/DL1/20210923_v0.7.5_prod5_trans_80_dynamic_cleaning/dl1_gamma_20deg_180deg_run202___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono_off0.4.h5"
# dl1_real_data_path = "/Users/thomasvuillaume/Work/CTA/Data/LST1/dl1_LST-1.Run09704.0000.h5"

dl1_mc_path = "../../share/data/MC_data/dl1_gamma_example.h5"
dl1_real_data_path = "../../share/data/real_data/dl1_realdata_example.h5"


In [ ]:
def read_camera_geometry(filename):
    return SubarrayDescription.from_hdf(filename).tel[1].camera.geometry

In [ ]:
mc_geom = read_camera_geometry(dl1_mc_path)

In [ ]:
real_geom = read_camera_geometry(dl1_real_data_path)

## Creating a fake image

In [ ]:
import astropy.units as u
from ctapipe.image import toymodel

model = toymodel.Gaussian(
    x=0.2 * u.m,
    y=0.0 * u.m,
    width=0.05 * u.m,
    length=0.15 * u.m,
    psi="35d",
)

image, sig, bg = model.generate_image(mc_geom, intensity=1500, nsb_level_pe=10)

## Display images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
CameraDisplay(mc_geom, image, ax=axes[0])
CameraDisplay(real_geom, image, ax=axes[1])

Both display look identical.

However, there are differences in the geometries in the camera rotation and pixels positions:

In [ ]:
mc_geom

In [ ]:
print("Pixels positions are equal?")
print(np.allclose(mc_geom.pix_x - real_geom.pix_x, 0, atol=1e-5) & np.allclose(mc_geom.pix_y - real_geom.pix_y, 0, atol=1e-5))

print("Pixels rotation angles:")
print(mc_geom.pix_rotation, real_geom.pix_rotation)

## Interpolating images with dl1_data_handler

In [ ]:
from copy import deepcopy
from dl1_data_handler.image_mapper import ImageMapper


def interpolate_image(geom, image, rotate=True):
    # The geometry needs to be rotated for the interpolation to work
    geom_ = deepcopy(geom)
    if rotate:
        geom_.rotate(geom_.pix_rotation)

    image_mapper = ImageMapper(
        pixel_positions={geom_.name: np.array([geom_.pix_x.to_value(u.m), geom_.pix_y.to_value(u.m)])},
        camera_types=[geom_.name],
        mapping_method={geom_.name: "bilinear_interpolation"},
        interpolation_image_shape={geom_.name: (55, 55, 1)},
    )

    interpolated_image = image_mapper.map_image(image.reshape(-1, 1), geom.name)
    return interpolated_image

In [ ]:
mc_interpolated_image = interpolate_image(mc_geom, image)
real_interpolated_image = interpolate_image(real_geom, image)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(mc_interpolated_image)
axes[1].imshow(real_interpolated_image)

### Without rotation

In [ ]:
mc_interpolated_image = interpolate_image(mc_geom, image, False)
real_interpolated_image = interpolate_image(real_geom, image, False)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(mc_interpolated_image)
axes[1].imshow(real_interpolated_image)

## Solving the issue by using a fixed standard geometry

In [ ]:
lstcam_geom = CameraGeometry.from_name("LSTCam")

In [ ]:
def reorganize_image(source_geom, target_geom, image):
    inj_table = source_geom.position_to_pix_index(target_geom.pix_x, target_geom.pix_y)
    reorganized_image = image[..., inj_table]
    return reorganized_image

In [ ]:
mc_reorganized_image = reorganize_image(mc_geom, lstcam_geom, image)
real_reorganized_image = reorganize_image(real_geom, lstcam_geom, image)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
CameraDisplay(lstcam_geom, mc_reorganized_image, ax=axes[0])
CameraDisplay(lstcam_geom, real_reorganized_image, ax=axes[1])

In [ ]:
mc_interpolated_image = interpolate_image(lstcam_geom, mc_reorganized_image)
real_interpolated_image = interpolate_image(lstcam_geom, real_reorganized_image)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(mc_interpolated_image)
axes[1].imshow(real_interpolated_image)

### Note: be mindful to NOT rotate the camera before pixel reorganization:

In [ ]:
lstcam_geom.rotate(lstcam_geom.pix_rotation)
mc_geom_ = deepcopy(mc_geom)
real_geom_ = deepcopy(real_geom)
mc_geom_.rotate(mc_geom_.pix_rotation)
real_geom_.rotate(real_geom_.pix_rotation)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
CameraDisplay(mc_geom_, image, ax=axes[0])
CameraDisplay(real_geom_, image, ax=axes[1])

# Test using ctapipe coordinates 

In [ ]:
from ctapipe.coordinates import CameraFrame, EngineeringCameraFrame, TelescopeFrame

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
mc_cam_geom = mc_geom.transform_to(CameraFrame())
mc_eng_geom = mc_geom.transform_to(EngineeringCameraFrame())
mc_tel_geom = mc_geom.transform_to(TelescopeFrame())

CameraDisplay(mc_cam_geom, image, ax=axes[0])
CameraDisplay(mc_eng_geom, image, ax=axes[1])
CameraDisplay(mc_tel_geom, image, ax=axes[2])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
real_cam_geom = real_geom.transform_to(CameraFrame())
real_eng_geom = real_geom.transform_to(EngineeringCameraFrame())

# Transformation to telescope frame failing
# real_tel_geom = real_geom.transform_to(TelescopeFrame())

CameraDisplay(real_cam_geom, image, ax=axes[0])
CameraDisplay(real_eng_geom, image, ax=axes[1])
# CameraDisplay(real_tel_geom, image, ax=axes[2])

## Interpolation

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

for i, geom in enumerate([mc_cam_geom, mc_eng_geom, mc_tel_geom]):
    try:
        mc_interpolated_image = interpolate_image(geom, image)
        axes[i].imshow(mc_interpolated_image)
    except:
        print(f"Fail for {i}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

for i, geom in enumerate([real_cam_geom, real_eng_geom, None]):
    try:
        interpolated_image = interpolate_image(geom, image)
        axes[i].imshow(interpolated_image)
    except:
        print(f"Fail for {i}")